# Homework 1 Part 4

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri
#### Notebooks Prepared By: Zeinab Sadat Taghavi, Hamed Jamshidian, Seyed Mohammad Reza Modarres

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between `## Your code begins ##` and `## Your code ends ##`) with the appropriate details.


## Introduction

In this notebook you have to use [Adapter Hub](https://docs.adapterhub.ml/overview.html) to define and train your adapters.

### Requirements

In [1]:
%%capture
! pip install datasets adapter-transformers

### Imports

In [2]:
from tqdm.notebook import tqdm
from IPython import display

import numpy as np
import pandas as pd
import math

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import T5TokenizerFast, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

### Constants

We will use `t5-small` as our base model from Hugging Face ([HF_Link](https://huggingface.co/t5-small)). And we use `32` as our adapter bottleneck size.

In [3]:
#####################################
###### DO NOT CHANGE THIS CELL ######
#####################################

BASE_MODEL_NAME = 't5-small'

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataset

### Load dataset

`imdb` dataset is a famouns NLP sentiment dataset. Each row of data is either `negative` or `positive`.

In [4]:
dataset = load_dataset('imdb')
dataset.pop('unsupervised')
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


### Define related functions

Because `T5` model is a sequence to sequence model we should map our labels to label_names before training and doing vice versa duing calculating metrics.

The functions `id2label` and `label2id` are defined to do this.

In [5]:
def id2label(ids):
    label_names = ['negative', 'positive']
    return [label_names[id] for id in ids]

def label2id(labels):
    label_names_dict = {
        'negative': 0,
        'positive': 1
    }
    return [
        label_names_dict.get(label, 2)
        for label in labels
    ]

# Tokenizer

### Load tokenizer

In [6]:
tokenizer = T5TokenizerFast.from_pretrained(BASE_MODEL_NAME)

### Process dataset using tokenizer

In this step we will getting our dataset ready for training.

We preprocess tokenize our `text` and `label`.

In [7]:
def preprocess_input(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    return text

def map_function(row):
    processed_input = [
        preprocess_input(text)
        for text in row['text']
    ]
    input_info = tokenizer(processed_input, truncation=True, max_length=256)
    output_info = tokenizer(id2label(row['label']))
    return {
        **input_info,
        'labels': output_info.input_ids
    }


dataset = dataset.map(map_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Model

### Load model and create adapter

In next part create `PfeifferConfig` by considering `BOTTLENECK_SIZE`.

Complete the next cell using methods of `train_adapter` and `add_adapter`.

Report final test data performance.s

Read [this page](https://docs.adapterhub.ml/training.html) for more information.

In [8]:
# from transformers import PfeifferConfig /// (has been moved to a stand alone package)
import adapters
from adapters import SeqBnConfig

BOTTLENECK_SIZE = 8
HIDDEN_SIZE = 512

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)
######### Your code begins #########
####################################
####################################
adapters.init(model)
config = SeqBnConfig(mh_adapter=False, output_adapter=True, reduction_factor=HIDDEN_SIZE/BOTTLENECK_SIZE, non_linearity='relu')
model.add_adapter('sharif_llm', config=config)
model.train_adapter("sharif_llm")

# Train and evaluate

### Train

Complete next part to train your peft using `AdapterTrainer`.

In [9]:
from transformers import TrainingArguments
from adapters import AdapterTrainer


col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

training_args = TrainingArguments(
    './temp',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='no',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_accumulation_steps=16,
    report_to='none'
)

######### Your code begins #########
####################################
####################################
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    data_collator=col_fn
    # compute_metrics=accuracy_score
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,0.661300,0.141083
2,0.152900,0.136690
3,0.144400,0.132570
4,0.139200,0.125679
5,0.135300,0.123938
6,0.133300,0.123896
7,0.132600,0.122475
8,0.128500,0.121550
9,0.127600,0.121606
10,0.126500,0.121724


TrainOutput(global_step=7820, training_loss=0.18816959449397327, metrics={'train_runtime': 3499.4669, 'train_samples_per_second': 71.439, 'train_steps_per_second': 2.235, 'total_flos': 1.695787008e+16, 'train_loss': 0.18816959449397327, 'epoch': 10.0})

### Evaluate

In [10]:
def _predict(model, row):
    return model.generate(
        input_ids=row.input_ids,
        attention_mask=row.attention_mask,
        max_length=5
    )

def tokenizer_ids_to_label(all_input_ids):
    return tokenizer.batch_decode(all_input_ids, skip_special_tokens=True)

def valid_loop(model, loader, compute_metrics):
    model.eval()

    all_true = []
    all_pred = []

    with torch.no_grad():
        for row in tqdm(loader, desc='Validating:'):
            row.to(model.device)
            pred = _predict(model, row)

            all_true += row.labels.detach().cpu().tolist()
            all_pred += pred.detach().cpu().tolist()

    all_true = label2id(tokenizer_ids_to_label(all_true))
    all_pred = label2id(tokenizer_ids_to_label(all_pred))

    return {'valid_acc': compute_metrics(y_true=all_true, y_pred=all_pred)}


test_loader = torch.utils.data.DataLoader(
    dataset['test'],
    batch_size=BATCH_SIZE,
    collate_fn=col_fn,
    shuffle=True
)

In [11]:
# trainer.evaluate()

valid_loop(model, test_loader, compute_metrics=accuracy_score)

Validating::   0%|          | 0/782 [00:00<?, ?it/s]

{'valid_acc': 0.9014}

# Bottleneck effect

Check the model performance with `BOTTLENECK_SIZE = 1` and report the number.

In [ ]:
from transformers import PfeifferConfig

BOTTLENECK_SIZE = 1

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

######### Your code begins #########
####################################
####################################